In [2]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import tensorflow as tf

# Load processed train-test datasets
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

# Load trained BiLSTM Model
bilstm_model = tf.keras.models.load_model("bilstm_alarm_model.h5")

# **Pastikan input memiliki format (samples, timesteps, features)**
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Extract LSTM Features for XGBoost
lstm_features_train = bilstm_model.predict(X_train)  # Sekarang dalam format (samples, 32)
lstm_features_test = bilstm_model.predict(X_test)

# Train XGBoost Classifier
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, objective='multi:softmax', num_class=4)
xgb_model.fit(lstm_features_train, y_train)

# Predict with XGBoost
y_pred_xgb = xgb_model.predict(lstm_features_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Model Accuracy (Using LSTM Features): {accuracy_xgb * 100:.2f}%")
print("Classification Report (XGBoost):\n", classification_report(y_test, y_pred_xgb))

# Save XGBoost Model
xgb_model.save_model("xgboost_lstm_model.json")
print("XGBoost model saved as xgboost_lstm_model.json")


4779/4779 ━━━━━━━━━━━━━━━━━━━━ 5s 888us/step
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  
XGBoost Model Accuracy (Using LSTM Features): 94.45%
Classification Report (XGBoost):
               precision    recall  f1-score   support

           0       0.99      0.90      0.95      9558
           1       0.80      0.95      0.87      1145
           2       0.78      0.99      0.87      2302
           3       0.99      0.99      0.99      6557

    accuracy                           0.94     19562
   macro avg       0.89      0.96      0.92     19562
weighted avg       0.95      0.94      0.95     19562

XGBoost model saved as xgboost_lstm_model.json
